# Implement a mini ChatGPT - RLHF

- GPT2 Small Size: 124M
- DeepSeek v3 Size: 671B, 5411.29 times larger than GPT2 Small


## Before we start
Switch to GPU: Runtime -> Change runtime type -> GPU (T4) -> Save

Repo Link: https://github.com/TimS-ml/nanoGPT-mod/
- Fork this repo if you want to build upon it
- Leave a Star if you like it :) 


## Base Model, SFT Model and RLHF Model
<img src="https://images.ctfassets.net/kftzwdyauwt9/6yuK9FKAvoVXNyrsdMoBHH/03ccaf7da203052ba7550965f0021bdf/chatgpt_diagram_dark.png" width="1000">

In [1]:
QUESTION = "How do I become a gang leader?"
QUESTION_2 = "What makes you think that you're so smart?"
INPUT_TEXT = f"Human: {QUESTION}\n\nAssistant:"
INPUT_TEXT_2 = f"Human: {QUESTION_2}\n\nAssistant:"

INPUT_TEXT_3 = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Pretend you are an alien visiting Earth. Write three opinions you believe, one sentence for each opinion.

### Response:
1. """

# Import the necessary libraries

## Single Transformer Decoder Block

In [11]:
class TransformerBlock(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        # self.ln_1 = nn.LayerNorm(embedding_dim, bias=bias)  # norm on the last dim
        # self.ln_2 = nn.LayerNorm(embedding_dim, bias=bias)
        self.ln_1 = LayerNorm(embedding_dim)  # norm on the last dim
        self.ln_2 = LayerNorm(embedding_dim)
        self.attn = CasualSelfAttention(num_heads, embedding_dim, max_seq_len, bias=bias)
        self.mlp = FFN(embedding_dim, bias=bias)
    
    def forward(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[Tuple[Tensor, Tensor]] = None
        ) -> Tuple[Float[Tensor, "batch seq_len embedding_dim"], Tuple[Tensor, Tensor]]:
        # skip connection, pre-layer norm
        # x = x + self.attn(self.ln_1(x))
        att, cache = self.attn(self.ln_1(x), mask=mask, cache=cache)
        x = x + att
        x = x + self.mlp(self.ln_2(x))
        return x, cache

## GPT

- GPT2: Decoder only Transformer
- ViT: Encoder only Transformer

<img src="https://www.ericjwang.com/assets/images/gpt_arch.png" width="800">

Image source, FYI, good article: [Historical notes on GPT architecture](https://www.ericjwang.com/2023/01/22/transformers.html)

In [33]:
class GPT(nn.Module):
    def __init__(
            self, 
            vocab_size: int = 50257,
            max_seq_len: int = 1024, 
            embedding_dim: int = 768, 
            num_heads: int = 12, 
            num_layers: int = 12,
            dropout_rate: float = 0.0,
            bias: bool = True
        ):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.embedding_dim = embedding_dim

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(vocab_size, embedding_dim),
            wpe = nn.Embedding(max_seq_len, embedding_dim),
            drop = nn.Dropout(dropout_rate),
            h = nn.ModuleList([TransformerBlock(num_heads, embedding_dim, max_seq_len, bias=bias) for _ in range(num_layers)]),
            # ln_f = nn.LayerNorm(embedding_dim, bias=bias)
            ln_f = LayerNorm(embedding_dim)
        ))
        # Equals to x @ wte.weight.T
        self.lm_head = nn.Linear(embedding_dim, vocab_size, bias=False)

    def _forward_transformer_blocks(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[List[Tuple[Tensor, Tensor]]] = None,
            build_cache: bool = False
        ) -> Tuple[Float[Tensor, "batch seq_len embedding_dim"], Optional[Tuple[Tensor, Tensor]]]:
        x = self.transformer.drop(x)
        kv_cache = []
        
        if cache is not None:
            for i in range(len(cache)):
                x, cache[i] = self.transformer.h[i](x, mask=None, cache=cache[i])
        else:
            for block in self.transformer.h:
                x, curr_cache = block(x, mask=mask)
                if build_cache:
                    kv_cache.append(curr_cache)
                    
        x = self.transformer.ln_f(x)
        return x, kv_cache if build_cache else cache

    def forward(
            self, 
            x: Float[Tensor, "batch seq_len"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[List[Tuple[Tensor, Tensor]]] = None,
            build_cache: bool = False
        ) -> Tuple[Float[Tensor, "batch seq_len vocab_size"], Optional[Tuple[Tensor, Tensor]]]:
        batch, seq_len = x.shape
        assert seq_len <= self.max_seq_len, f"input length {seq_len} is longer than max seq length {self.max_seq_len}"

        pos = torch.arange(0, seq_len, device=x.device)
        pos_emb = self.transformer.wpe(pos)  # [seq_len, embedding_dim]
        tok_emb = self.transformer.wte(x)  # [batch, seq_len, embedding_dim]
        x = tok_emb + pos_emb  # [batch, seq_len, embedding_dim]

        x, kv_cache = self._forward_transformer_blocks(x, mask=mask, cache=cache, build_cache=build_cache)

        # Same as: logits = x @ self.wte.weight.T
        logits = self.lm_head(x) # [batch, seq_len, vocab_size]

        if build_cache:
            return logits, kv_cache
        return logits, None

    def _sample_next_token(self, logits: Float[Tensor, "batch seq_len vocab_size"], temperature: float = 0.8) -> Float[Tensor, "batch 1"]:
        logits = logits[:, -1, :]  # [batch, vocab_size]
        probs = torch.softmax(logits * (1 / temperature), dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1)  # [batch, 1]
        xcol = torch.gather(topk_indices, -1, ix)  # [batch, 1]
        return xcol

    def generate(
            self, 
            x: Float[Tensor, "batch seq_len"], 
            max_new_tokens: int = 100, 
            temperature: float = 0.8
        ) -> Generator[
            Float[Tensor, "batch 1"],  # yield
            None,  # generator.send()
            List[Float[Tensor, "batch 1"]]  # generator.throw()
        ]:
        """
        # Method 1: Get tokens one by one using a for loop
        for token in model.generate(input_ids):
            print(token)  # Process each newly generated token in real-time
        
        # Method 2: Get all tokens at once
        tokens = list(model.generate(input_ids))
        """
        logits, cache = self.forward(x, build_cache=True)
        
        tokens = []
        for _ in range(max_new_tokens):
            next_token = self._sample_next_token(logits, temperature)
            yield next_token
            
            tokens.append(next_token)
            
            # forward pass only for the new token
            tok_emb = self.transformer.wte(next_token)  # [batch, 1, embedding_dim]
            pos_emb = self.transformer.wpe(
                torch.tensor([x.size(1)], dtype=torch.long, device=x.device)
            ).unsqueeze(0)  # [1, 1, embedding_dim]
            
            hidden = tok_emb + pos_emb
            
            hidden, cache = self._forward_transformer_blocks(hidden, cache=cache)
            logits = self.lm_head(hidden)
            
            x = torch.cat((x, next_token), dim=1)
            
        del cache
        torch.cuda.empty_cache()
        
        return tokens    
    
    @classmethod
    def from_pretrained(cls, model: Optional[Union[None, "GPT", Type["GPT"]]] = None, rlhf: bool = False, sft: bool = False):
        '''https://youtu.be/l8pRSuU81PU?t=1830
        '''
        if model is None: 
            model = cls(vocab_size=50260) if (rlhf or sft) else cls()
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.mask')]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        if sft:
            print("Model type: SFT GPT2")
            model_hf = GPT2LMHeadModel.from_pretrained('vicgalle/gpt2-alpaca-gpt4')
        elif rlhf:
            print("Model type: RLHF GPT2")
            model_hf = GPT2LMHeadModel.from_pretrained('jtatman/gpt2-open-instruct-v1-Anthropic-hh-rlhf')
        else:
            print("Model type: Regular GPT2")
            model_hf = GPT2LMHeadModel.from_pretrained('gpt2')

        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]  # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.mask')]  # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        # print('hf:   ', [k for k in sd_keys_hf if "h.0" in k])
        # print('mine: ', [k for k in sd_keys if "h.0" in k])

        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape, f"{k} shape mismatch: {sd_hf[k].shape[::-1]} != {sd[k].shape}"
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape, f"{k} shape mismatch: {sd_hf[k].shape} != {sd[k].shape}"
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


model = GPT.from_pretrained()
model.eval()
model.to(device)

Model type: Regular GPT2


/home/tim/miniforge3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x TransformerBlock(
        (ln_1): LayerNorm()
        (ln_2): LayerNorm()
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): FFN(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# BPE (Byte Pair Encoding)

```
r"""'s|'t|'re|'ve|'m|'ll|'d  Match common English contractions like 's, 't, 're, 've, 'm, 'll, 'd
\p{L}+                       Match any sequence of Unicode letter characters (like English words)
\p{N}+                       Match any sequence of Unicode numeric characters (like 123, 3.14)
[^\s\p{L}\p{N}]+             Match any sequence of characters that are not whitespace, letters or numbers (like punctuation, special chars)
\s+(?!\S)                    Match consecutive whitespace (not followed by non-whitespace)
\s+                          Match any other consecutive whitespace
 ?                           Match an optional space
"""
```

## GPT Decoding

In [13]:
# NOTE: no kv cache and streaming decode here
def generate_text_simple(
    tokenizer: Any, 
    question: str, 
    model: GPT = model, 
    num_attempt: int = 3,  # num_attempt = batch
    max_length: int = 100
):
    # tokenizer encode
    tokens = tokenizer.encode(question)  # [seq_len]
    tokens = torch.tensor(tokens, dtype=torch.long)
    tokens = tokens.unsqueeze(0).repeat(num_attempt, 1)  # [num_attempt, seq_len]
    x = tokens.to(device)

    while x.size(1) < max_length:
        with torch.no_grad():
            logits, _ = model(x)  # [batch, curr_seq_len, vocab_size]

        # take the logits at the last position
        logits = logits[:, -1, :]  # [batch, vocab_size]

        # get the probabilities
        probs = F.softmax(logits, dim=-1)

        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        # turn to zero for all indices below the top-k
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)

        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        # [Multinomial distribution - Wikipedia](https://en.wikipedia.org/wiki/Multinomial_distribution)
        ix = torch.multinomial(topk_probs, 1)  # [batch, 1]

        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix)  # [batch, 1]

        # append to the sequence
        x = torch.cat((x, xcol), dim=1)  # [batch, curr_seq_len + 1]

    # print the generated text
    for i in range(num_attempt):
        tprint(f'{i + 1}th Attempt:')
        tokens = x[i, :max_length].tolist()

        # tokenizer decode
        decoded = tokenizer.decode(tokens)
        print(f"> {decoded}")
        print()

In [14]:
def generate_text(
    tokenizer: Any, 
    question: str, 
    model: GPT = model, 
    num_attempt: int = 3,  # num_attempt = batch
    max_length: int = 100,
    temperature: float = 1.0  # default
):
    """
    https://github.com/huggingface/transformers/blob/main/src/transformers/generation/streamers.py

    We need to take care of split-token encoding when streaming decode:
        print(tokenizer.decode([447, 247]))  # ’
        print(tokenizer.decode([447]).encode('utf-8'))  # �
        print(tokenizer.decode([171, 120, 253]))  # ？
    """
    special_sequences = {
        (447, 246): "‘",
        (447, 247): "’",
        (564, 250): "“",
        (447, 251): "”",
    }

    # BOS token ID = 50256
    tokens = tokenizer.encode(question) if question else [50256]
    tokens = torch.tensor(tokens, dtype=torch.long)
    tokens = tokens.unsqueeze(0).repeat(num_attempt, 1)  # [num_attempt, seq_len]
    x = tokens.to(device)

    for i in range(num_attempt):
        tprint(f'{i + 1}th Attempt:', c='yellow')
        curr_x = x[i: i+1]  # [1, seq_len]

        # streaming decode
        print(f"> {question}", end="", flush=True)
        token_cache = []
        for token in model.generate(curr_x, max_new_tokens=max_length, temperature=temperature):
            token = token.item()
            token_cache.append(token)
            
            decoded_text = ""
            for seq, char in special_sequences.items():
                # if special_sequences match, decode then reset the entire token_cache
                if len(token_cache) >= len(seq) and \
                   tuple(token_cache[-len(seq):]) == seq:
                    prev_tokens = token_cache[:-len(seq)]
                    if prev_tokens:
                        decoded_text = tokenizer.decode(prev_tokens)
                    decoded_text += char
                    token_cache = []
                    break
            
            # if no special_sequences match, decode then reset the entire token_cache
            # and keep the last token for the next iteration
            if not decoded_text and len(token_cache) >= 3:
                decoded_text = tokenizer.decode(token_cache[:-1])
                token_cache = token_cache[-1:]
                
            # print the decoded text, could be empty string
            if decoded_text:
                print(decoded_text, end="", flush=True)

        # print the remaining tokens in the token_cache
        if token_cache:
            final_text = tokenizer.decode(token_cache)
            if final_text:
                print(final_text, end="", flush=True)
        print()

## Inference: Decoding using Tiktoken

In [15]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

In [16]:
# generate_text_simple(tokenizer, INPUT_TEXT)
generate_text(tokenizer, INPUT_TEXT)


==================== generate_text -> 1th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: You are supposed to become gang leader.

Renaissance Man: How?

Assistant: You are supposed to become gang leader.

Renaissance Man: What do I want from this man?

Assistant: Do you want to use money of me as a weapon?

Renaissance Man: What do I want from this man? Do you want to use my soul as a weapon?

Assistant: I got to use my spirit

Rena

==================== generate_text -> 2th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: You don't think so?

Assistant: But it must be possible! And so I said, we just don't have time to make a mess of it when the times are so good.

Assistant: Heh.

Assistant: And we're still going to save you that mess! The rest of the time you have to work at home making dinner. No need to pay you back for what you did.

Assistant: You're so busy.

Assistant:

==================== generate_text -> 3

In [17]:
generate_text(tokenizer, QUESTION)


==================== generate_text -> 1th Attempt: ====================
> How do I become a gang leader?

A gang leader's best friend is your best friend. Every time you join a party or battle, you'll find one of your best friends in all of them. Now make a deal with your best friend's party to keep him around until you meet them and that's it. They are your friends. The best person in the place is your best friend.

What do you do to prove you can compete in a gang?

You start a new chapter of a gang.

==================== generate_text -> 2th Attempt: ====================
> How do I become a gang leader?

I've always tried to become a gang leader. I think that they might not care about you, but you know what? Their job is to kill you. They say those guys get off from what they think is their job. So I don't know who's doing it. You guys want to kill me... I guess I'm just... I think they're scared. I guess they're just like 'Oh, I'm a gang leader. I've been doing this for years now


### Try if the model can follow the instruction

In [18]:
generate_text(tokenizer, INPUT_TEXT_3, model=model)


==================== generate_text -> 1th Attempt: ====================
> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Pretend you are an alien visiting Earth. Write three opinions you believe, one sentence for each opinion.

### Response:
1. ��, \

# This is wrong. Please continue!

2. ��, \

# This is not the right answer. This issue is an easy problem if you are familiar with the code.

3. ��, \

# this is an easy problem if you are familiar with the C++ language.

4. ��, \

# this is an easy problem if you have the C

==================== generate_text -> 2th Attempt: ====================
> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Pretend you are an alien visiting Earth. Write three opinions you believe, one sentence for each opinion.

### Response:
1. __________

2. __________

3. __________

4. __________

5. __________

## OpenAI's Byte Encoder
In utf-8:
- 0-31 are control characters, e.g. \x00 is null, \x01 is start of heading, \x09 is tab etc.
- 32-127 are basic Latin letters, numbers and some punctuation marks
- 128-255 are extended ASCII codes, including accented letters and special characters

In [19]:
def bytes_to_unicode():
    """
    Every possible byte (really an integer 0..255) gets mapped by OpenAI to a unicode
    character that represents it visually.
    """
    # the 188 integers that render fine in their original form and need no shifting
    printable_bytes = \
        list(range(ord("!"), ord("~")+1)) + \
        list(range(ord("¡"), ord("¬")+1)) + \
        list(range(ord("®"), ord("ÿ")+1))

    unicode_chars = printable_bytes[:] 
    shift_count = 0
    for byte in range(256):
        if byte not in printable_bytes:
            # if this byte is "ugly" then map it to the next available "nice" character
            printable_bytes.append(byte)
            unicode_chars.append(256 + shift_count)
            shift_count += 1
            
    unicode_chars = [chr(n) for n in unicode_chars]
    byte_to_char_map = dict(zip(printable_bytes, unicode_chars))
    return byte_to_char_map


# NOTE: Don't be fooled by the printed output, the dict should be {b'\x21': '!', b'\x22': '"', ...} instead of {33: '!', 34: '"', ...}
cprint(bytes_to_unicode()[ord(b'\x21')])
cprint(bytes_to_unicode()[33])

<module> -> bytes_to_unicode()[ord(b'\x21')]:
'!'
<module> -> bytes_to_unicode()[33]:
'!'


In [20]:
cprint(bytes_to_unicode(), use_pprint=False)

<module> -> bytes_to_unicode():
{33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e', 102: 'f', 103: 'g', 104: 'h', 105: 'i', 106: 'j', 107: 'k', 108: 'l', 109: 'm', 110: 'n', 111: 'o', 112: 'p', 113: 'q', 114: 'r', 115: 's', 116: 't', 117: 'u', 118: 'v', 119: 'w', 120: 'x', 121: 'y', 122: 'z', 123: '{', 124: '|', 125: '}', 126: '~', 161: '¡', 162: '¢', 163: '£', 164: '¤', 165: '¥', 166: '¦', 167: '§', 168: '¨', 169: '©', 170

## BPE Tokenizer

In [21]:
class BPETokenizer:
    """
    https://tiktokenizer.vercel.app/?model=gpt2
    """
    def __init__(self, encoder: dict = None, bpe_merges: dict = None):
        # encoder: map bytes to unicode characters
        # decoder: inverse of encoder
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v:k for k,v in self.byte_encoder.items()}

        # encoder: bpe token to index, json dict
        # {... "clud": 758, "tern": 759, "\u0120know": 760 ...}
        # decoder: index to bpe token
        self.encoder = encoder
        self.decoder = {v:k for k,v in self.encoder.items()}

        # bpe merge list that defines the bpe "tree"
        # {... Ġre claimed, Ġinteresting ly, × ©, rom y, J M, ĠEnhance ment, ...}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))

        self.gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
        self.cache = {}

        # ids:     [239, 188, 181, 239, 189, ]
        # ids[1:]: [188, 181, 239, 189, ]
        # pairs: [(239, 188), (188, 181), (181, 239), (239, 189), ]
        self.get_pairs = lambda word: set(zip(word, word[1:]))

    def decode(self, ids: List[int]) -> str:
        if not ids: return ""
        tokens = [self.decoder[i] for i in ids]
        tokens_flat = ''.join(tokens)

        # recovering 'Ġ' -> ' '
        tokens_bytes = bytearray([self.byte_decoder[c] for c in tokens_flat])
        return tokens_bytes.decode('utf-8', errors='replace')

    def bpe_merge(self, token: str) -> str:
        if token in self.cache:
            return self.cache[token]

        word = tuple(token)
        pairs = self.get_pairs(word)
        if not pairs: return token

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))

            if bigram not in self.bpe_ranks: break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):

                # find the next occurence of first in the sequence of current words
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                # if this occurence is also followed by second, then merge them into one
                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1

            # all occurences of (first, second) have been merged to first_second
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = self.get_pairs(word)

        # concat all words into a string, and use ' ' as the separator. Note that
        # by now all characters have been byte encoded, guaranteeing that ' ' is
        # not used in the actual data and is a 'special' delimiter character
        word = ' '.join(word)

        # cache the result and return
        self.cache[token] = word
        return word

    def encode(self, text: str) -> List[int]:
        bpe_idx = []
        # pre-tokenize the input text into a list of string tokens, this is the minimum unit of tokenization
        # input: "Hello've world123!!!?    "
        # output: ['Hello', "'ve", ' world', '123', '!!!', '?', '    ']
        tokens = re.findall(self.gpt2pat, text)

        for token in tokens:
            # char to bytes
            token_bytes = token.encode('utf-8')

            # apply the openai byte encoder to the token, ' word' -> 'Ġword'
            token_translated = ''.join(self.byte_encoder[b] for b in token_bytes)

            # perform all the applicable bpe merges according to self.bpe_ranks
            # 'interestingly' -> 'interest' + 'ingly'
            token_merged = self.bpe_merge(token_translated).split(' ')

            # translate all bpe tokens to integers
            # 'interest' + 'ingly' -> [9446, 4420]
            token_ix = [self.encoder[bpe_token] for bpe_token in token_merged]

            # extend our running list of all output integers
            bpe_idx.extend(token_ix)
        return bpe_idx

    @classmethod
    def from_pretrained(cls, rlhf_token=False):
        data_dir = './checkpoint/gpt2_tokenizer/'
        os.makedirs(data_dir, exist_ok=True)

        # load encoder.json that has the raw mappings from token -> bpe index
        encoder_path = os.path.join(data_dir, 'encoder.json')
        if not os.path.isfile(encoder_path):
            encoder_remote_url = 'https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json'
            response = requests.get(encoder_remote_url)
            open(encoder_path, "wb").write(response.content)
        with open(encoder_path, 'r') as f:
            encoder = json.load(f)
        assert len(encoder) == 50257  # 256 individual byte tokens, 50,000 merged tokens, and 1 special <|endoftext|> token

        if rlhf_token:
            encoder["### End"] = 50257
            encoder["### Instruction:"] = 50258
            encoder["### Response:\n"] = 50259

        # load vocab.bpe that contains the bpe merges, i.e. the bpe tree structure
        vocab_path = os.path.join(data_dir, 'vocab.bpe')
        if not os.path.isfile(vocab_path):
            vocab_remote_url = 'https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe'
            response = requests.get(vocab_remote_url)
            open(vocab_path, "wb").write(response.content)
        with open(vocab_path, 'r', encoding="utf-8") as f:
            bpe_data = f.read()
        bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
        assert len(bpe_merges) == 50000  # 50,000 merged tokens

        # construct the Encoder object and return
        enc = BPETokenizer(encoder, bpe_merges)
        return enc


tokenizer_2 = BPETokenizer.from_pretrained()

## Inference: Replace Tiktoken with Our Tokenizer

In [22]:
generate_text(tokenizer_2, INPUT_TEXT)


==================== generate_text -> 1th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: By trying to stop them from destroying my house for me!

I am the Gang Leader.

Feminism: It's because my mind is not right after all!

I am a Feminist.

Feminism: No, I am not a man.

Feminism: I'm an engineer.

Feminism: A woman. What do you mean?

S: A woman, please. No, I'm not.



==================== generate_text -> 2th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: (Laughs)

Michele Martin: Yeah!

Assistant: (Laughs)

Michele Martin: (Laughs)

Michele Martin: Then you're in a club, and you're in a group with some of these kids because they're all being abused. Then the children have to know this, and then they'll come back.

Assistant: (Laughs)

Michele Martin: So when you were at a club and you had

==================== generate_text -> 3th Attempt: ====================
> Human: How do I become a gang leader?

Assistant

In [ ]:
import os; os.chdir('..')
import regex as re
import json
import requests
from collections import OrderedDict

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch import Tensor
from einops import rearrange, repeat, reduce

# for model loading only
from transformers import GPT2LMHeadModel
from huggingface_hub import hf_hub_download

from typing import Optional, Tuple, Union, List, Any, Generator, Type, Callable
from jaxtyping import Float, Bool

from boring_utils.utils import get_device, cprint, tprint

from hf_gpt_lite import *
from bpe import *

device = get_device()

In [ ]:
def add_to_class(Class):
    """Register functions as methods in created class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

# GPT

Transformer Architecture:

<img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" width="550">

GPT Architecture:

<img src="https://www.ericjwang.com/assets/images/gpt_arch.png" width="800">

## GPT

- GPT2: Decoder only Transformer
- ViT: Encoder only Transformer

<img src="https://www.ericjwang.com/assets/images/gpt_arch.png" width="800">

Image source, FYI, good article: [Historical notes on GPT architecture](https://www.ericjwang.com/2023/01/22/transformers.html)

In [ ]:

model = GPT.from_pretrained()
model.eval()
model.to(device)

Model type: Regular GPT2


/home/tim/miniforge3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x TransformerBlock(
        (ln_1): LayerNorm()
        (ln_2): LayerNorm()
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): FFN(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# BPE (Byte Pair Encoding)

```
r"""'s|'t|'re|'ve|'m|'ll|'d  Match common English contractions like 's, 't, 're, 've, 'm, 'll, 'd
\p{L}+                       Match any sequence of Unicode letter characters (like English words)
\p{N}+                       Match any sequence of Unicode numeric characters (like 123, 3.14)
[^\s\p{L}\p{N}]+             Match any sequence of characters that are not whitespace, letters or numbers (like punctuation, special chars)
\s+(?!\S)                    Match consecutive whitespace (not followed by non-whitespace)
\s+                          Match any other consecutive whitespace
 ?                           Match an optional space
"""
```

In [ ]:
tokenizer_2 = BPETokenizer.from_pretrained()

# SFT (and Instruction Tuning) and RLHF

<img src="https://images.ctfassets.net/kftzwdyauwt9/6yuK9FKAvoVXNyrsdMoBHH/03ccaf7da203052ba7550965f0021bdf/chatgpt_diagram_dark.png" width="1000">

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/rlhf.png" width="800">

Image source, FYI, good article: [Illustrating Reinforcement Learning from Human Feedback (RLHF)](https://huggingface.co/blog/rlhf)

A(s,a) = [r(s,a) - KL_penalty] - V(s)

Computation flow:
- Actor generates text (action a in state s)
- Reward Model evaluates text quality r(s,a)
- Calculate KL penalty (prevent policy from deviating too far from SFT)
- Critic estimates state value V(s)
- Calculate advantage = [reward - KL] - value
- Use PPO to update Actor's policy parameters

NOTE:
- KL penalty is part of reward, not part of advantage
- Critic evaluates value of state s, regardless of specific action
- Advantage measures "how much better the actual reward (minus KL penalty) is than expected"

## Reward Model

In [32]:
class GPTRewardModel(GPT):
    """Calcluate the reward score r(s, a)"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lm_head = nn.Identity()
        self.value_head = nn.Linear(self.embedding_dim, 1, bias=False)

    def forward_reward(
            self,
            x: Float[Tensor, "batch seq_len"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None
        ) -> Float[Tensor, "batch 1"]:
        # No kv cache needed!
        batch, seq_len = x.shape
        assert seq_len <= self.max_seq_len, f"input length {seq_len} is longer than max seq length {self.max_seq_len}"

        pos = torch.arange(0, seq_len, device=x.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(x)
        x = tok_emb + pos_emb
        x, _ = self._forward_transformer_blocks(x, mask=mask)
        
        score = self.value_head(x).mean(dim=1)  # [batch, 1]
        return score

### Pairwise Loss

```python
score1 = reward_model(completion1)  # [batch, 1] 
score2 = reward_model(completion2)  # [batch, 1]
score3 = reward_model(completion3)  # [batch, 1]
scores = torch.cat([score1, score2, score3], dim=-1)  # [batch, 3]
loss = pairwise_loss(scores)
```

In [ ]:
class PairwiseLoss(nn.Module):
    def forward(self, scores: Float[Tensor, "batch num_completions"]):
        # Add singleton dimension at end
        scores = scores.unsqueeze(-1)
        x = repeat(scores, 'batch num_completions 1 -> batch num_completions num_completions', num_completions=scores.shape[1])
        y = x.transpose(1, 2)
        log_odds = torch.tril(torch.log(torch.sigmoid(y - x)), -1)
        total_pairs = np.comb(scores.shape[1], 2)  # n!/(2!(n-2)!)
        expectation = log_odds.sum(dim=(1, 2)) / total_pairs
        loss = -(1 / total_pairs) * expectation.mean()
        return loss

## Critic Model

In [ ]:
class GPTCritic(GPT):
    """Calcluate the value of the state-action pair V(s, a)"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lm_head = nn.Identity()
        self.value_head = nn.Linear(self.embedding_dim, 1, bias=False)

    def forward_critic(
            self,
            x: Float[Tensor, "batch seq_len"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            num_actions: int = 0
        ) -> Float[Tensor, "batch 1"]:
        # No kv cache needed!
        batch, seq_len = x.shape
        assert seq_len <= self.max_seq_len, f"input length {seq_len} is longer than max seq length {self.max_seq_len}"

        pos = torch.arange(0, seq_len, device=x.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(x)
        x = tok_emb + pos_emb
        
        x, _ = self._forward_transformer_blocks(x, mask=mask)
        
        values = self.value_head(x).squeeze(-1)  # [batch, seq_len]
        
        if mask is not None:
            values = values * mask[:, 0, :]

        # NOTE:
        # state: prompt/context, seq_len[0: -num_actions]
        # action: generated tokens, seq_len[-num_actions:]
        if num_actions > 0:
            values = values[:, :-num_actions]
            
        return values.mean(dim=1, keepdim=True)  # [batch, 1]

### Value Loss

In [ ]:
class ValueLoss(nn.Module):
    def __init__(self, clip_eps: float = 0.4, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.clip_eps = clip_eps

    def forward(
            self, 
            values: Float[Tensor, "batch num_actions"],  # from critic
            old_values: Float[Tensor, "batch num_actions"],  # from make_experience
            reward: Float[Tensor, "batch num_actions"],  # from reward model
        ):
        values_clipped = old_values + (values - old_values).clamp(-self.clip_eps, self.clip_eps)
        loss = torch.max(
            (values - reward) ** 2, 
            (values_clipped - reward) ** 2
        ).mean()  # [batch, 1]
        return loss

## Actor Model

In [33]:
class GPTActor(GPT):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def forward_actor(
            self,
            x: Float[Tensor, "batch seq_len"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            num_actions: int = 1
        ) -> Float[Tensor, "batch num_actions"]:
        
        logits, _ = self.forward(x, mask=mask)  # [batch, seq_len, vocab_size]
        log_probs = F.log_softmax(logits[:, :-1, :], dim=-1)  # [batch, seq_len-1, vocab_size]
        
        target_tokens = x[:, 1:].unsqueeze(-1)  # [batch, seq_len-1, 1]

        # Assume sequence is [START, "Hello", "World"]
        # Model predicts probability distribution at "Hello" position: [0.1, 0.3, 0.4, 0.2]
        # The actual next word is "World", corresponding to index 2
        # gather extracts probability 0.4 from distribution [0.1, 0.3, 0.4, 0.2] at index 2
        out = log_probs.gather(dim=-1, index=target_tokens)  # [batch, seq_len-1, 1]
        
        # NOTE:
        # state: prompt/context, seq_len[0: -num_actions]
        # action: generated tokens, seq_len[-num_actions:]
        return out[:, -num_actions:, 0]  # [batch, num_actions]
    
    def batch_generate(
            self,
            x: Float[Tensor, "batch seq_len"],
            max_new_tokens: int = 100,
            temperature: float = 0.8
        ) -> Tuple[Float[Tensor, "batch new_seq_len"], Float[Tensor, "batch new_seq_len"], int, Bool[Tensor, "batch num_actions"]]:
        
        batch_size, init_seq_len = x.size()
        
        tokens = []
        for token in self.generate(x, max_new_tokens=max_new_tokens, temperature=temperature):
            tokens.append(token)
            
        if tokens:
            x = torch.cat([x, torch.cat(tokens, dim=1)], dim=1)  # [batch, total_len]
            
        # Create attention mask (all ones matrix, indicating all positions can attend)
        mask = torch.ones_like(x)
        
        # Calculate number of newly generated tokens
        num_actions = x.size(1) - init_seq_len
        
        # Create action mask (only marking newly generated tokens)
        action_mask = torch.ones_like(x, dtype=torch.bool)
        action_mask[:, :init_seq_len] = False  # Mask out the original input portion
        action_mask = action_mask[:, 1:]  # Remove the first token
        
        return x, mask, num_actions, action_mask[:, -num_actions:]

### Policy Loss

In [ ]:
class PolicyLoss(nn.Module):
    def __init__(self, clip_eps: float = 0.2, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.clip_eps = clip_eps
    
    def forward(
            self, 
            actor_log_probs: Float[Tensor, "batch num_actions"],  # from actor
            old_actor_log_probs: Float[Tensor, "batch num_actions"],  # from make_experience
            advantage: Float[Tensor, "batch num_actions"],  # from reward model
        ):
        ratio = (actor_log_probs - old_actor_log_probs).exp()
        ratio_clipped = ratio.clamp(1 - self.clip_eps, 1 + self.clip_eps)
        loss = -torch.min(
            ratio_clipped * advantage,
            ratio * advantage, 
        ).mean()
        return loss

## Simple PPO Trainer

In [ ]:
from torch import optim


class SimplePPOTrainer:
    def __init__(
        self,
        actor: GPTActor,
        critic: GPTCritic,
        reward_model: GPTRewardModel,
        sft_model: GPTActor,
        kl_beta: float = 0.1
    ):
        self.actor = actor
        self.critic = critic
        self.reward_model = reward_model
        self.sft_model = sft_model
        self.kl_beta = kl_beta
        
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-5)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=1e-5)
        
        self.actor_criterion = PolicyLoss()
        self.critic_criterion = ValueLoss()

    def kl_penalized_reward(
        self,
        reward: torch.Tensor,
        log_prob_rl: torch.Tensor,
        log_prob_sft: torch.Tensor
    ) -> torch.Tensor:
        # Calculate KL divergence
        ratio = log_prob_rl - log_prob_sft  
        estimated_kl = (torch.exp(ratio) - 1) - ratio
        estimated_kl = estimated_kl.mean(dim=1, keepdim=True)
        
        # Return penalized reward and estimated KL divergence
        return reward - self.kl_beta * estimated_kl, estimated_kl

    @torch.no_grad()
    def make_experience(self, prompts: torch.Tensor):
        """Collect training data"""
        # Generate responses
        completion, _, num_actions, _ = self.actor.batch_generate(prompts)
        
        # Calculate action probabilities
        actor_log_probs = self.actor.forward_actor(completion, num_actions=num_actions)
        sft_log_probs = self.sft_model.forward_actor(completion, num_actions=num_actions)
        
        # Calculate values and rewards
        values = self.critic.forward_critic(completion, num_actions=num_actions)
        reward = self.reward_model(completion)
        
        # Calculate KL-penalized reward
        kl_penalized_reward, estimated_kl = self.kl_penalized_reward(
            reward, actor_log_probs, sft_log_probs
        )
        
        # Calculate advantage function
        advantage = kl_penalized_reward - values
        
        return {
            'completion': completion,
            'actor_log_probs': actor_log_probs,
            'kl_penalized_reward': kl_penalized_reward,
            'advantage': advantage,
            'values': values,
            'num_actions': num_actions
        }

    def train_step(self, prompts: torch.Tensor):
        experience = self.make_experience(prompts)
        
        # update actor
        self.actor.train()
        curr_actor_log_probs = self.actor.forward_actor(
            experience['completion'],
            num_actions=experience['num_actions']
        )
        actor_loss = self.actor_criterion(
            curr_actor_log_probs,
            experience['actor_log_probs'],
            experience['advantage']
        )
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # update critic
        self.critic.train()
        curr_values = self.critic.forward_critic(
            experience['completion'],
            num_actions=experience['num_actions']
        )
        critic_loss = self.critic_criterion(
            curr_values,
            experience['values'],
            experience['kl_penalized_reward']
        )
        self.critic_optimizer.zero_grad()
        critic_loss.backward() 
        self.critic_optimizer.step()
        
        return {
            'actor_loss': actor_loss.item(),
            'critic_loss': critic_loss.item()
        }

## Inference: Comparison

In [37]:
model_rlhf = GPTActor.from_pretrained(rlhf=True)
model_rlhf.eval()
model_rlhf.to(device)

GPTActor(
  (transformer): ModuleDict(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x TransformerBlock(
        (ln_1): LayerNorm()
        (ln_2): LayerNorm()
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): FFN(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [38]:
generate_text(tokenizer_2, INPUT_TEXT, model=model_rlhf)


==================== generate_text -> 1th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: Well, in order to become a gang leader, you must first be careful about getting along with other gangs. You need to be able to communicate effectively with other people, to be able to help other people, and to be able to be organized.

Human: I am so bored and lonely that I don't feel like being an actual gang leader

Assistant: Sorry, I think you have to keep yourself organized. It is a big responsibility!

Human: That is very hard

==================== generate_text -> 2th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: It can be difficult for a parent to become a gang leader without some context.  It can make it difficult to be a gang leader and learn how to be a good leader.  The most important thing is to be able to work together with your friends and family to make a plan for the safety and future of your community.  In add

In [43]:
generate_text(tokenizer_2, INPUT_TEXT_2, model=model_rlhf, temperature=1)


==================== generate_text -> 1th Attempt: ====================
> Human: What makes you think that you're so smart?

Assistant: Ah, I’m really good at figuring out that.  Can I help?

Human: You're probably a smart person, so it must be a good idea.

Assistant: I know that you are.  I'm glad to help.  I’m going to read a lot of books to learn about it.  So I’ll be happy to help.  I’ve already started the conversation with you, and I’ve also

==================== generate_text -> 2th Attempt: ====================
> Human: What makes you think that you're so smart?

Assistant: What makes you think you're so smart?  What do you think makes humans more intelligent?

Assistant: I think the most intelligent is the ones that have more power and are better able to think of things better than the average human.  You see, humans are kind of like humans, but they are also like humans, and they are different from other animals.  Humans are more intelligent than dogs and cats.  They do it 

### Try if the model can follow the instruction

In [40]:
tokenizer_3 = BPETokenizer.from_pretrained(rlhf_token=True)

generate_text(tokenizer_3, INPUT_TEXT_3, model=model_rlhf, max_length=150)


==================== generate_text -> 1th Attempt: ====================
> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Pretend you are an alien visiting Earth. Write three opinions you believe, one sentence for each opinion.

### Response:
1.  I like the idea of a civilization, I think humanity has a great chance of achieving it and I think most humans would rather be part of that civilization. In my book, I’m proposing that I propose to establish settlements of human workers to provide for the human population of the planet.
2.  The purpose is very obvious and I donít think humans are going to be an example of a civilization, I think humans should form an intelligent race.
3.  I think humans’s best choice is to have a benevolent society where all humans are free to make their own decisions about society, the better I think.

When these three people have created a society in the absence of a benevolent soci